<a href="https://colab.research.google.com/github/yoshitha1209/Machine-Learning/blob/main/23003_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

file_path='/content/drive/MyDrive/Colab Notebooks/Copy of training_mathbert(1).xlsx'
df=pd.read_excel(file_path)
df.dropna(inplace=True)
X = df.drop(columns=['output'])
X = pd.get_dummies(X)
y = df['output']
y = y.astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf=RandomForestClassifier()
param_dist={
    'n_estimators':[100,200,300],
    'max_depth':[None,10,20,30],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
    'bootstrap':[True,False]
}

random_search=RandomizedSearchCV(estimator=rf,param_distributions=param_dist,n_iter=20,cv=5,verbose=2,random_state=42,n_jobs=-1)
random_search.fit(X_train,y_train)

best_params=random_search.best_params_
print('Best parameters:',best_params)

best_model=random_search.best_estimator_
y_pred=best_model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print('Accuracy:',accuracy)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20, 'bootstrap': True}
Accuracy: 0.14601769911504425
